In [1]:
import numpy as np
from typing import List, Optional, Dict, Set, Callable, Any
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets

from cross_validation import cv_tslearn, print_cv_results
from eval_on_test import validate_tslearn, print_test_results
from utils import load_from_pickle, save_to_pickle

# Enumerate all tslearn datasets

In [ ]:
# _datasets = [
#             'ArticularyWordRecognition', 
#             'BasicMotions', 
#             'Cricket',
#             #'ERing',
#             'Libras', 
#             'NATOPS', 
#             'RacketSports',     
#             'FingerMovements',
#             'Heartbeat',
#             'SelfRegulationSCP1', 
#             'UWaveGestureLibrary'
#             ]

# import tslearn
# UCR_UEA_datasets = tslearn.datasets.UCR_UEA_datasets()

# for dataset_name in UCR_UEA_datasets.list_multivariate_datasets():
# #for dataset_name in _datasets:
#     print("Dataset:", dataset_name)
#     dataset = UCR_UEA_datasets.load_dataset(dataset_name)
#     if dataset[0] is not None:
#         X_train, y_train, X_test, y_test = dataset
#         num_classes = len(np.unique(y_train))
#         N_train, T, d = X_train.shape
#         N_test, _, _  = X_test.shape
        
#         print("Number of Classes:", num_classes)
#         print("Dimension of path:", d)
#         print("Length:", T)
#         print("Train Size, Test Size", N_train, N_test)
#         print()
#     else:
#         print("No dataset found")
#         print()

#yes
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

#yes
# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

#yes
# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

#why not
# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

#longLength
# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

#big
# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

#yes
# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

#why not, maybe big length
# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

#smallTrain
# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

#yes
# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

#big
# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

#yes
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

#TODO I SHOULD INCLUDE
# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

#length
# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

#yes
# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

#yes
# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

#TODO SHOULD INCLUDE highDim
# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

#NO, dim=1, big length, large num classes
# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

#yes
# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

#yes
# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

#NO, long, also very small set
# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

#yes
# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# Cross Validation on Train

In [2]:
cv_results = cv_tslearn(
    dataset_names = [
        #'ArticularyWordRecognition', 
        #'BasicMotions', 
        #'Cricket',
             #########'ERing', #cant find dataset
        'Libras', 
        #'NATOPS', 
        #'RacketSports',     
        #'FingerMovements',
        #'Heartbeat',
        #'SelfRegulationSCP1', 
        #'UWaveGestureLibrary',
        #'PenDigits',
        #'LSST',
        #'EthanolConcentration',
        ],
    kernel_names = [
        "linear",
        #"rbf",
        #"poly",
        #"gak",
        #"truncated sig",
        #"truncated sig rbf",
        #"signature pde rbf",
        #"integral linear",
        #"integral rbf",
        #"integral poly",
        ],
        k=5,
        n_repeats=1,
        n_jobs_repeats=1,
        n_jobs_gram=1,
        verbose=False,
        )

Dataset: Libras
Number of Classes: 15
Dimension of path: 2
Length: 45
Train: 180
Test: N/A


Label for linear: 100%|██████████| 15/15 [00:03<00:00,  4.19it/s]

Time taken for kernel linear: 3.5873541831970215 seconds
Time taken for dataset Libras: 3.589930534362793 seconds





##### Print CV results

In [3]:
print_cv_results(cv_results)

Cross Validation Results
Libras
Number of Classes: 15
Dimension of path: 2
Length: 45
Train: 180
Test: N/A

linear
final_score_avgs 1.7629168762848821
params_score_avgs [1.763]
thresh_score_avgs [0.941 1.156 1.398 1.482 1.564 1.632 1.641 1.549 0.861]
1
{'threshold': 7}
10
{'threshold': 8}
11
{'threshold': 2}
12
{'threshold': 7}
13
{'threshold': 4}
14
{'threshold': 6}
15
{'threshold': 3}
2
{'threshold': 8}
3
{'threshold': 8}
4
{'threshold': 7}
5
{'threshold': 7}
6
{'threshold': 6}
7
{'threshold': 8}
8
{'threshold': 8}
9
{'threshold': 7}

End dataset 





# Validate on Test

In [8]:
test_results = validate_tslearn(cv_results, n_jobs=4, verbose=False)

Start validation on test sets
FingerMovements
Number of Classes: 2
Dimension of path: 28
Length: 50
Train: 316
Test: 100
Kernel: linear


100%|██████████| 2/2 [00:00<00:00, 11.68it/s]


Kernel: rbf


100%|██████████| 2/2 [00:00<00:00, 10.10it/s]


Kernel: poly


...

Kernel: integral rbf


100%|██████████| 8/8 [00:00<00:00, 28.52it/s]


Kernel: integral poly


100%|██████████| 8/8 [00:00<00:00, 31.30it/s]

Total elapsed time for UWaveGestureLibrary: 1298.941909919 seconds

End validation on test sets





##### Print test results

In [10]:
print_test_results(test_results)

Test Results

Dataset: FingerMovements
Number of Classes: 2
Dimension of path: 28
Length: 50
Train: 316
Test: 100

Kernel: linear
Conformance AUC: 0.507
Mahalanobis AUC: 0.522
Conformance PR AUC: 0.516
Mahalanobis PR AUC: 0.514

Kernel: rbf
Conformance AUC: 0.505
Mahalanobis AUC: 0.452
Conformance PR AUC: 0.528
Mahalanobis PR AUC: 0.502

Kernel: poly
Conformance AUC: 0.556
Mahalanobis AUC: 0.518
Conformance PR AUC: 0.559
Mahalanobis PR AUC: 0.521

Kernel: gak
Conformance AUC: 0.537
Mahalanobis AUC: 0.486
Conformance PR AUC: 0.522
Mahalanobis PR AUC: 0.5

Kernel: truncated sig
Conformance AUC: 0.487
Mahalanobis AUC: 0.479
Conformance PR AUC: 0.509
Mahalanobis PR AUC: 0.508

Kernel: truncated sig rbf
Conformance AUC: 0.457
Mahalanobis AUC: 0.457
Conformance PR AUC: 0.52
Mahalanobis PR AUC: 0.511

Kernel: signature pde rbf
Conformance AUC: 0.444
Mahalanobis AUC: 0.459
Conformance PR AUC: 0.504
Mahalanobis PR AUC: 0.495

Kernel: integral linear
Conformance AUC: 0.503
Mahalanobis AUC: 0.523

# Read CV data from file and print results

In [7]:
def read_dicts_from_pickle(paths:List[str]) -> Dict:
    dicts = [load_from_pickle(path)
             for path in paths]
    joined_dicts = {}
    for d in dicts:
        joined_dicts.update(d)
    return joined_dicts

# Load the cross validation results
cv_results = read_dicts_from_pickle(
    [
    #"../Data/cv_ArticularyWordRecognition.pkl",
    #"../Data/cv_BasicMotions.pkl",
    #"../Data/cv_EthanolConcentration.pkl",
    "../Data/cv_FingerMovements.pkl",
    #"../Data/cv_Heartbeat.pkl",
    "../Data/cv_Libras.pkl",
     "../Data/cv_NATOPS.pkl",
    # "../Data/cv_RacketSports.pkl", 
    # "../Data/cv_SelfRegulationSCP1.pkl",
     "../Data/cv_UWaveGestureLibrary.pkl",
    ])
print_cv_results(cv_results)

Cross Validation Results
FingerMovements
Number of Classes: 2
Dimension of path: 28
Length: 50
Train: 316
Test: N/A

linear
final_score_avgs 1.1305557310915748
params_score_avgs [1.131]
thresh_score_avgs [1.075 1.073 1.113 1.113 1.102 1.101 1.099 1.1   1.087 1.093 1.088 1.08
 1.065 1.063 1.049 1.031 1.034 1.043 1.054 1.067 1.07  1.064 1.055 1.064
 1.068 1.075 1.077 1.076 1.072 1.071 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.   ]
left
{'threshold': 1}
right
{'threshold': 7}

rbf
final_sco

In [ ]:
test_results = read_dicts_from_pickle(["TODO"])
print_test_results(test_results)